In [1]:
# Runs:
#   train0a: first model, same parameters for all images
#   train0b: first model, special parameters
#   train1a: second model, same parameters for all images
#   train1b: second model, special parameters

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# test plot

fig, ax = plt.subplots(layout="constrained")

df = pd.read_csv("cache/train0a_analyze_data.csv")
days = df["days"].to_numpy()
img_count = df["img_count"].to_numpy()

ax.scatter(days, img_count)
ax.plot(days, img_count)

ax.set(xlabel="Days",
       ylabel="Leaf Count")

plt.show()
fig.savefig("exp_plots/test.png")


In [ ]:
# train general
import analyze

def plot(days, img_count, save_name):
    fig, ax = plt.subplots(layout="constrained")

    #ax.scatter(days, img_count)
    ax.plot(days, img_count, marker='o')

    ax.set(xlabel="Days",
           ylabel="Leaf Count")
           #ylim=(min(img_count), 6000))

    #plt.show()
    fig.savefig(f"exp_plots/{save_name}.png")


def multi_plot(datas, names, cols, rows, save_name: str = "alltogether"):
    multi_fig, axs = plt.subplots(cols, rows, figsize=(15,12), gridspec_kw={"wspace": 0.05}, layout="constrained")

    count = 0
    for data, ax in zip(datas, axs.flat):
        x, y = data

        #ax.scattercount)
        ax.plot(x, y, marker='o')
        
        # label as (0a), (0b)... on top-left corner
        ax.text(0.05, 0.95, f"({names[count]})", transform=ax.transAxes, 
                fontsize=14, verticalalignment='top')
        count += 1

        ax.set(xlabel="Days",
               ylabel="Leaf Count",
               xlim=(x[0]-1, x[-1] + 1))
        
    multi_fig.savefig(f"exp_plots/{save_name}.png")



trains = ["0a", "0b", "1a", "1b"]
train_data = []
for train in trains:
    data = analyze.leaf_analyze(use_cached=True, cache_file=f"cache/train{train}_analyze_data.csv")
    print(data, type(data))
    train_data.append(data)


# individual plot
for train, data in zip(trains, train_data):
    days, img_count = data
    plot(days, img_count, save_name="train" + train)

# 2x2 grid plot
multi_plot(train_data, trains, 2, 2)


plt.show()


In [ ]:
# testing more types of plots
def average_plot(datas, save_name: str = "avg_plot"):
    # get x values (same for every data: days in range 1..12)
    x_vals = datas[0][0]

    # create a matrix where each column is a x value and each row is the correspondent y value accross all data
    data_matrix = np.empty(shape=(len(datas), len(x_vals)))
    for row, (x, y) in enumerate(datas):
        x_indices = np.searchsorted(x_vals, x)
        data_matrix[row, x_indices] = y

    # calculate each column average and std deviaton (corresponds to the average/std count of the day)
    avg_data = np.mean(data_matrix, axis=0)
    std_err = np.std(data_matrix, axis=0)
    

    fig, ax = plt.subplots(layout="constrained")
    ax.plot(x_vals, avg_data, marker='o')

    ax.errorbar(x_vals, avg_data, yerr=std_err, c="b",)

    ax.set(xlabel="Days",
           ylabel="Average Count")

    fig.savefig(f"exp_plots/{save_name}.png")



def polar_plot(days, counts):
    theta = np.linspace(0, 2*np.pi, len(days))
    r = counts

    fig, ax = plt.subplots()
    ax.plot(theta, r)

def stem_plot(days, counts):
    fig, ax = plt.subplots()
    ax.stem(days, counts)

def quiver_plot(days, counts):
    u = np.diff(days)
    v = np.diff(counts)
    fig, ax = plt.subplots()
    ax.quiver(days[:-1], counts[:-1], u, v)

def multi_quiver_plot(datas, names, cols, rows, save_name: str = "quiver_alltogether"):
    multi_fig, axs = plt.subplots(cols, rows, figsize=(15,12), gridspec_kw={"wspace": 0.05}, layout="constrained")

    idx = 0
    for (days, counts), ax in zip(datas, axs.flat):
        u = np.diff(days)
        v = np.diff(counts)

        ax.quiver(days[:-1], counts[:-1], u, v)

        ax.text(0.05, 0.95, f"({names[idx]})", transform=ax.transAxes, 
                fontsize=14, verticalalignment='top')
        idx += 1

        ax.set(xlabel="Days",
               ylabel="Leaf Count")

    multi_fig.savefig(f"exp_plots/{save_name}.png")


a0, b0, a1, b1 = train_data
#average plot 0a and 0b
average_plot([a0, b0], save_name="avg_plot_0a0b")

#average plot 1a and 1b
average_plot([a1, b1], save_name="avg_plot_1a1b")

#average plot 0a and 1a
average_plot([a0, a1], save_name="avg_plot_0a1a")

#average plot 0b and 1b
average_plot([b0, b1], save_name="avg_plot_0b1b")

#average plot 0a to 1b
average_plot(train_data, save_name="avg_plot_0ato1b")


train1 = train_data[1]
polar_plot(train1[0], train1[1])
stem_plot(train1[0], train1[1])

multi_quiver_plot(train_data, trains, 2, 2)
plt.show()